#### Libraries

In [1]:
# http://www.eecs.harvard.edu/~michaelm/CS222/assignment2.pdf

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
plt.rcParams["figure.figsize"] = [16,9]

#### Problem 8: JPG

In [4]:
# Inputs
img = [[124, 125, 122, 120, 122, 119, 117, 118],
       [120, 120, 120, 119, 119, 120, 120, 120],
       [125, 124, 123, 122, 121, 120, 119, 118],
       [125, 124, 123, 122, 121, 120, 119, 118],
       [130, 131, 132, 133, 134, 130, 126, 122],
       [140, 137, 137, 133, 133, 137, 135, 130],
       [150, 147, 150, 150, 150, 150, 150, 150],
       [160, 160, 162, 164, 168, 170, 172, 175]]
qnt = [[16, 11, 10, 16, 24,  40,  51,   61],
       [12, 12, 14, 19, 26,  58,  60,   55],
       [14, 13, 16, 24, 40,  57,  69,   56],
       [14, 17, 22, 29, 51,  87,  80,   62],
       [18, 22, 37, 56, 68,  109, 103,  77],
       [24, 35, 55, 64, 81,  104, 113,  92],
       [49, 64, 78, 87, 103, 121, 120, 101],
       [72, 92, 95, 98, 112, 100, 103,  99]]
img = np.array(img)
qnt = np.array(qnt)

# Define 2d DCT
from scipy.fftpack import dct, idct
def dct_2d(block, inverse=False):
    if inverse: return idct(idct(block.T, norm='ortho').T, norm='ortho')
    else: return dct(dct(block.T, norm='ortho').T, norm='ortho')

# Transform
cmp = img - 128 # center 
cmp = dct_2d(cmp) # dct
cmp = cmp / qnt # quantize
cmp = cmp.round().astype(int) # round down

# Decompress 
dec = cmp.astype(float) * qnt
dec = dct_2d(dec, inverse=True)
dec = dec.round().astype(int) + 128

if False: print(cmp.tolist(), dec.tolist())

#### Delta encoding

In [6]:
# Parameters
N = 10000
b = 30
R = 2**b

def get_nums(N, R):
    '''Generate random numbers'''
    nums = np.random.randint(0, R, size=N, dtype=np.uint32)
    nums = np.sort(nums)
    return nums

def get_difs(N, R):
    '''Calculate differences'''
    nums = get_nums(N, R)
    first_char = nums[0]
    difs = nums[1:] - nums[:-1]
    return difs
    
# Part 1
trials = []
for _ in range(10):
    difs = get_difs(N, R)
    k = np.ceil(np.log2(np.max(difs)))
    space = b + np.ceil(np.log2(b)) + (N-1) * k
    trials.append(space)
print([int(x) for x in trials])

[200015, 200015, 200015, 210014, 210014, 210014, 210014, 200015, 200015, 210014]


In [264]:
# Part 2: Helper functions
def encode(input_string):
    '''Run length encoding'''
    count = 1
    prev = ''
    lst = []
    for character in input_string:
        if character != prev or count > 127:
            if prev:
                entry = (prev,count)
                lst.append(entry)
                #print lst
            count = 1
            prev = character
        else:
            count += 1
    else:
        entry = (character,count)
        lst.append(entry)
        return lst
 
def decode(lst):
    q = ""
    for character, count in lst:
        q += character * count
    return q

In [275]:
# Part 2 
N = 10000
R = 2**30
m = 10
m = 30 - m

# 32 bits in order [0..7][8..15][16..23][24..31] --> 
def shuffle(x):   return np.concatenate((x[:,24:32], x[:,16:24], x[:,8:16], x[:,0:8]), axis=1)
def unshuffle(x): return np.concatenate((x[:,24:32], x[:,16:24], x[:,8:16], x[:,0:8]), axis=1)

# 32-bit integers to bits
for i in range(10):
    nums = get_nums(N, R)
    bits = np.unpackbits(nums.view(np.uint8)).reshape(N, 32)
    bits = shuffle(bits)
    bits_lower = bits[:,-m:]
    bits_upper = bits[:,:-m]
    bits_upper = bits_upper[:,2:] # remove 
    # we can recover our original bits with:
    # bits = unshuffle(np.concatenate((bits_upper, bits_lower), axis=1))
    
    # Convert upper bits to string and run-length encode
    string_upper = str(list(bits_upper.T.flatten()))[1:-1].replace(' ','').replace(',','')
    string_upper_encoded = encode(string_upper)
    
    # Compute number of bytes needed to send run-length encoded upper bits
    bits_per_rle_element = np.ceil(1 + np.log2(1+max(x[1] for x in string_upper_encoded)))
    upper_send = bits_per_rle_element * len(string_upper_encoded)
    lower_send = bits_lower.size
    
    # Total number of bits
    total_send = upper_send + lower_send + 4
    print(total_send)

222216.0
222180.0
222207.0
222225.0
222198.0
222234.0
222216.0
222207.0
222225.0
222198.0
